In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
from functools import partial
import pandas as pd

In [2]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Response', 'Response_status', 'Time (seconds)'])

In [3]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [4]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

In [5]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Response':result.text, 'Response_status':result.status, 'Time (seconds)':f'{execution_time:.6f}'}
        df.append(data, ignore_index=True)
        print(f"Execution time of {result}: {execution_time:.6f} seconds - Response===> {_.text} \n\n")
        return result, _
    return calculate_execution_time

In [6]:
def save_report(file_path, df):
    file_exists = os.path.exists(file_path)
    df.to_csv(file_path, index=False)
#     with open(file_path, 'a', newline='') as csv_file:
#         csv_writer = csv.writer(csv_file)
#         if not file_exists:
#             csv_writer.writerow(data[0])
#         for row in data[1:]:
#             csv_writer.writerow(row)

In [7]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        return response
    except Exception as e:
        print(e)
        

#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines
    prompts = read_payloads(request_payload_file)
    for i in range(5):
        for line_content in prompts:
        print(f"Prompt===> {line_content}") 
        payload['prompt'] = line_content
#             thread_func = partial(inference, payload_data=payload)
        with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
            futures = [executor.submit(process_text, text) for text in texts]
            executor.map(thread_func, range(1))
    save_report(report_path, df)

Line 0: {"payload":"Problem: Write a function in python to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n."}
Execution time of Thread 0: 4.644881 seconds - Response===> {"id":"cmpl-0a2c2858dfa64733869166942eea49d8","object":"text_completion","created":423720,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\nn = int(input(\"Enter a non-negative integer: \"))\nprint(\"The factorial of\", n, \"is\", factorial(n))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":118,"completion_tokens":73}} 


Execution time of Thread 2: 4.646673 seconds - Response===> {"id":"cmpl-df016cca9d674ae4924dc2f965530b9d","object":"text_completion","created":423720,"model":"/llmmodels/models--ise-uiuc--Magicode

Execution time of Thread 3: 2.408934 seconds - Response===> {"id":"cmpl-9d225c54001b4ff6ba3c5068d0289b8e","object":"text_completion","created":423725,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\ndef reverse_string(s):\n    return s[::-1]\n\nprint(reverse_string(\"Hello World\"))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":36,"total_tokens":68,"completion_tokens":32}} 

Execution time of Thread 2: 2.436452 seconds - Response===> {"id":"cmpl-01999321920d4332a5a029999bc9ddc4","object":"text_completion","created":423725,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\ndef reverse_string(s):\n    return s[::-1]\n\nprint(reverse_string(\"Hello World\"))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":36,"total_tokens":68,"completion_tokens":32}} 



Execution 

Execution time of Thread 14: 10.071283 seconds - Response===> {"id":"cmpl-95df92e6ca564a14be7a3fd72eb27fb4","object":"text_completion","created":423728,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef is_prime(num):\n    if num <= 1:\n        return False\n    elif num <= 3:\n        return True\n    elif num % 2 == 0 or num % 3 == 0:\n        return False\n    i = 5\n    while i * i <= num:\n        if num % i == 0 or num % (i + 2) == 0:\n            return False\n        i += 6\n    return True\n\n# Test the function\nprint(is_prime(2))  # Output: True\nprint(is_prime(4))  # Output: False\nprint(is_prime(17))  # Output: True\nprint(is_prime(100))  # Output: False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":207,"completion_tokens":165}} 


Execution time of Thread 6: 10.108638 seconds - Response===> {"id":"cmpl-6a9eb6bcb33c426ab351bb997fd668b0",

Execution time of Thread 3: 6.278432 seconds - Response===> {"id":"cmpl-413727da8b9848db868bbac835c73894","object":"text_completion","created":423742,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef sum_of_digits(num):\n    return sum(int(digit) for digit in str(num))\n\n# Test the function\nprint(sum_of_digits(1234))  # Output: 10\nprint(sum_of_digits(5678))  # Output: 26\nprint(sum_of_digits(9012))  # Output: 10","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":151,"completion_tokens":103}} 


Execution time of Thread 8: 6.182769 seconds - Response===> {"id":"cmpl-25e7dd91265e4e4da5082d97a1724af6","object":"text_completion","created":423742,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef sum_of_digits(num):\n    return sum(int(digit) for digit in str

Execution time of Thread 0: 6.605038 seconds - Response===> {"id":"cmpl-12027056073c416db91e722b451d4800","object":"text_completion","created":423748,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef fibonacci(n):\n    if n <= 0:\n        return \"Input should be a non-negative integer.\"\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        a, b = 0, 1\n        for _ in range(n - 2):\n            a, b = b, a + b\n        return b\n\nprint(fibonacci(10))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":158,"completion_tokens":110}} 


Execution time of Thread 3: 6.579113 seconds - Response===> {"id":"cmpl-4cf276d6b5d94b0095d69caece5fb9ad","object":"text_completion","created":423748,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"te

...

Execution time of Thread 0: 0.307209 seconds - Response===> {"id":"cmpl-79d5079107e2429d8add7c15a4314137","object":"text_completion","created":423785,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":49,"completion_tokens":1}} 


Execution time of Thread 2: 0.381883 seconds - Response===> {"id":"cmpl-d79c8abbccd243bbb61bf509938516c2","object":"text_completion","created":423785,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":49,"completion_tokens":1}} 


Execution time of Thread 5: 0.358487 seconds - Response===> {"id":"cmpl-fb6952b806ed47a1ab38530748cc2429","object":"text_completion","created":423785,"model":"/llmmodels/models--ise-uiuc--

Execution time of Thread 1: 0.207770 seconds - Response===> {"id":"cmpl-72ac0011457c4c008fba8caf8fb58f1b","object":"text_completion","created":423787,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":52,"total_tokens":53,"completion_tokens":1}} 


Execution time of Thread 0: 0.433723 seconds - Response===> {"id":"cmpl-adff8386d6bf47c596930b4b63233dfc","object":"text_completion","created":423787,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":52,"total_tokens":53,"completion_tokens":1}} 


Execution time of Thread 6: 0.408408 seconds - Response===> {"id":"cmpl-06f07c1d94bf4dc0bf35d745442c7865","object":"text_completion","created":423787,"model":"/llmmodels/models--ise-uiuc--

Execution time of Thread 2: 0.659961 seconds - Response===> {"id":"cmpl-cb615f5ee09b4159b8d53d04d5b7c721","object":"text_completion","created":423788,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":46,"total_tokens":47,"completion_tokens":1}} 


Execution time of Thread 13: 0.392040 seconds - Response===> {"id":"cmpl-28400027b05240c4ae21206b72d8d355","object":"text_completion","created":423788,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":46,"total_tokens":47,"completion_tokens":1}} 


Execution time of Thread 7: 0.585634 seconds - Response===> {"id":"cmpl-42a0e3df5a7f4bb5a12f948e1fd0e0aa","object":"text_completion","created":423788,"model":"/llmmodels/models--ise-uiuc-

Execution time of Thread 0: 11.676454 seconds - Response===> {"id":"cmpl-4635d3fb464741329b82d8ad56e30aa9","object":"text_completion","created":423790,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n#include <iostream>\n#include <string>\n\nstd::string reverseString(const std::string& s) {\n    std::string reversed = s;\n    int length = s.length();\n\n    for (int i = 0; i < length / 2; i++) {\n        char temp = reversed[i];\n        reversed[i] = reversed[length - i - 1];\n        reversed[length - i - 1] = temp;\n    }\n\n    return reversed;\n}\n\nint main() {\n    std::string s;\n    std::cout << \"Enter a string: \";\n    std::getline(std::cin, s);\n\n    std::string reversed = reverseString(s);\n    std::cout << \"Reversed string: \" << reversed << std::","logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":37,"total_tokens":237,"completion_tokens":200}} 


Execution 

KeyboardInterrupt: 

In [7]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))

{'id': 'cmpl-9142a552a9a1470e8ac53e85e51b2080', 'object': 'text_completion', 'created': 87834, 'model': '/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66', 'choices': [{'index': 0, 'text': '\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams("listen", "silent"))  # True\nprint(are_anagrams("hello", "world"))  # False', 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 45, 'total_tokens': 148, 'completion_tokens': 103}}
--- 5.502794504165649 seconds ---
